In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [3]:
def get_target(df, target_name):
    return df.drop(target_name, axis=1), df[target_name]

def drop_correlated_features(df, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df = df.drop(to_drop, axis=1)
    return df

def drop_min_rows(df, *values):
    for value in values:
        df = df.drop(df[df['DP'] == value].index, axis=0)
    return df

def balance_classes(df):
    df_balanced = pd.DataFrame()
    min_class_count = df['DP'].value_counts().min()
    for label in df['DP'].unique():
        df_label = df[df['DP'] == label]
        if len(df_label) > min_class_count:
            df_random_sample = df_label.sample(min_class_count)
            df_balanced = pd.concat([df_balanced, df_random_sample])
        else:
            df_balanced = pd.concat([df_balanced, df_label])
    return df_balanced

In [4]:
from posixpath import split
# load dataset

df = pd.read_csv("Data.csv", header = 0, delimiter = ";")
print("drop duplicates")
df = df.drop_duplicates()
constant_features = [column for column in df.columns if df[column].nunique() == 1]
print("drop constant features = ",len(constant_features))
df.drop(columns=constant_features, inplace=True)
print("split features and target")

df_features, df_target = get_target(df, 'DP')

df_corr = drop_correlated_features(df_features, 0.7)

df = pd.concat([df_corr, df_target], axis=1)
new_df = drop_min_rows(df, -60, -140)

new_df = balance_classes(new_df)

df_check = new_df.copy()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2

kbest = SelectKBest(k=100, score_func=mutual_info_classif)
kbest.fit(df_check.drop('DP', axis=1), df_check['DP'])
rec = kbest.get_support()
print(rec)
list_best = []
for i, r in enumerate(rec):
    if r == True:
        list_best.append(df_check.columns[i])
        print(df_check.columns[i], kbest.scores_[i])
if 'DP' in list_best:
    list_best.remove('DP')
df_best = df_check[list_best]

# check that dataframe contain column

if df_best.Disbalance.empty == None:
    df_best =  pd.concat([df_check.Disbalance, df_best], axis=1)

df_best.head()

drop duplicates
drop constant features =  1361
split features and target


/home/art/prog/MLUVrepo/envml/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=100 is greater than n_features=59. All the features will be returned.
  warnings.warn(


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True]
Disbalance 1.1799754537433969
ON_LINE gen 2 0.1359352161288241
QG gen 2 0.0
ON_LINE gen 3 0.08812063860109287
PG gen 5 1.1173659176335444
PG gen 27 0.428097055919757
QG gen 27 0.5817584848098201
QN node 7005 0.651446394762007
PN node 7008 0.30725956820375355
QN node 7011 0.30620069614825196
U node 7012 0.6258167482317618
QN node 7015 0.5536095024750556
QN node 7023 0.42801320097331885
QN node 7029 0.4260459498410776
QN node 7032 0.3834973403971609
QN node 7033 0.3535994376796463
QN node 7042 0.17921581691567123
QN node 7044 0.45266051624461623
QN node 7057 0.17913020262825485
QN node 7064 0.0
PN node 7065 0.7621705603546749
QN node 

,Disbalance,ON_LINE gen 2,QG gen 2,ON_LINE gen 3,PG gen 5,PG gen 27,QG gen 27,QN node 7005,PN node 7008,QN node 7011,...,QN node 7776,QN node 7819,QN node 7821,PN node 7829,PN node 7957,Q_BEG line 52,P_BEG line 77,Q_BEG line 207,P_BEG line 220,P_BEG line 317
5,143.14,0,0.0,0,0.0,0.0,0.0,0.660812,5.126475,0.484510,...,1.131414,0.0,0.180033,0.336362,0.0,0.08,-0.06,0.0,0.0,0.81
15,142.77,0,0.0,0,0.0,0.0,0.0,0.625786,5.078537,0.429343,...,1.121359,0.0,0.229031,0.326488,0.0,0.09,-0.24,0.0,0.0,0.49
16,142.60,0,0.0,0,0.0,0.0,0.0,0.703919,5.017258,0.473474,...,1.121407,0.0,0.227000,0.326575,0.0,0.09,-0.21,0.0,0.0,0.00
18,143.20,0,0.0,0,0.0,0.0,0.0,0.619212,5.051795,0.423491,...,1.101456,0.0,0.000000,0.325609,0.0,0.08,-0.30,0.0,0.0,0.00
19,142.49,0,0.0,0,0.0,0.0,0.0,0.685193,5.116674,0.466858,...,1.101435,0.0,0.000000,0.297376,0.0,0.08,-0.26,0.0,0.0,0.00


In [6]:
# Tensorflow Multiclass Classification
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# data for tensorflow model
data_labels = df_check["DP"]
data_labels = pd.factorize(data_labels)[0]
data_features = df_best
train_features, test_features, train_labels, test_labels = train_test_split(data_features, data_labels, test_size = 0.2, random_state = 42)

# make model

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(data_features.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.fit(train_features, train_labels, epochs=1000, batch_size=100)

accuracy = model.evaluate(test_features, test_labels)[1]
print("Accuracy: {:.2f}%".format(accuracy * 100))



Epoch 1/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - accuracy: 0.2246 - loss: 2.3309 
Epoch 2/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.6007 - loss: 1.0767
Epoch 3/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.7308 - loss: 0.7326
Epoch 4/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.7246 - loss: 0.6317
Epoch 5/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - accuracy: 0.7697 - loss: 0.5536
Epoch 6/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 0.7825 - loss: 0.4938
Epoch 7/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.8112 - loss: 0.4391
Epoch 8/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.8333 - loss: 0.4181
Epoch 9/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - accuracy: 0.8437 - loss: 0.3851
Epoch 10/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.8608 - loss: 0.3479
Epoch 11/1000
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.8577 - loss: 0.3568
Epoch 12/1000
17/17 ━━━━━━━━━